In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import requests

data = pd.read_excel("tickets_export.xlsx")
data.head()

,Nummer,Erstellt,Lösungszeit,Meldeberechtigter,Organisation,Anfrager,Kanal,Lieferant,Serviceangebot,Aktualisiert,...,Unterkategorie,Status,Dringlichkeit,Priorität,Abteilung,Ort,Auswirkung,Gelöst,SLA def processing breached,SLA def reaction breached
0,INC475189,44773.541250,1162838.0,"Widmer Manus, JUSTICE-STAW-BJS-Biel",JUSTICE,NaN,Service Portal,NaN,SOF: Support Applikation Tribuna Anonymisierun...,44787.000023,...,Software,Geschlossen,Mittel,Tief,Staatsanwaltschaft Berner Jura-Seeland,"Spitalstrasse 11, 2501 Biel-Bienne",mehrere Arbeitsplätze,44773.692975,False,False
1,INC475187,44773.432211,826745.0,"Saxer Vera, GSI-ZSHKK","Gesundheits-, Sozial- und Integrationsdirektion",NaN,Service Portal,NaN,SOF-CS: Profile,44783.001019,...,Benutzer Administration,Geschlossen,Tief,Tief,Zentrum für Sozial- und Heilpädagogik Köniz-Ke...,"Landorfstrasse 94, 3098 Köniz",einzelner Arbeitsplatz,44775.291354,False,False
2,INC475186,44773.226042,1967711.0,"Lecomte Fanny, SID-AJV-RGBI-B",Sicherheitsdirektion,"Tedde Tamara, SID-AJV-RGMO",Service Portal,NaN,SOF: Fachapplikation nicht definiert (SID-AJV),44796.000475,...,Benutzer Administration,Geschlossen,Tief,Tief,AJV Regionalgefängnis Moutier,"Rue du Château 30b, 2740 Moutier",einzelner Standort,44788.649132,False,False
3,INC475185,44772.958310,1089274.0,"Schwander Sarah, SID-AJV-RGBU-SL",Sicherheitsdirektion,NaN,Service Portal,NaN,SOF-CS: Profile,44785.565648,...,Software,Geschlossen,Tief,Mittel,AJV_RGBU Sicherheit / Loge,"Dunantstrasse 9, 3400 Burgdorf",mehrere Standorte,44781.378044,False,False
4,INC475184,44772.912060,871685.0,"Lecomte Fanny, SID-AJV-RGBI-B",Sicherheitsdirektion,NaN,Service Portal,NaN,SOF: Support Applikation GINA (SID),44783.001007,...,System Administration,Geschlossen,Tief,Tief,AJV_RGMO Betreuung,"Rue du Château 30b, 2740 Moutier",einzelner Standort,44775.332801,False,False


In [65]:
# remove NaNs and clean data
data["Betreff"] = data["Betreff"].fillna("").replace("\n", " ").replace("\r", " ").replace("\t", " ").replace("-", " ")
data["Beschreibung"] = data["Beschreibung"].fillna("").replace("\n", " ").replace("\r", " ").replace("\t", " ").replace("-", " ")
freetext_columns = ["Betreff", "Beschreibung"]

data["freetext"]  = data[freetext_columns].apply(lambda x: " ".join(x), axis=1)

# get stopwords from online source
stopwords = requests.get("https://raw.githubusercontent.com/stopwords-iso/stopwords-de/master/stopwords-de.txt").text.split("\n")

tfidf = TfidfVectorizer(stop_words=stopwords)
tfidf.fit(data["Betreff"] + " " + data["Beschreibung"])
index_dict = {v: k for k, v in tfidf.vocabulary_.items()}

In [73]:
num_top_words = 3

for i, row in data.iterrows():

    print("")
    print("Ticket", i+1)
    print("Top words per document:")
    # extract indices of top words
    top_words = sorted(
        list(enumerate(tfidf.transform([row["freetext"]]).toarray()[0])),
        key=lambda x: x[1],
        reverse=True,
    )[:num_top_words]

    # print top words
    for word_index, word_score in top_words:
        print(index_dict[word_index])

    # break to only print first ticket
    break


Ticket 1
Top words per document:
container_xp
tribuna
applikation
